In [ ]:
# Import dependencies
import json
import pandas as pd
import numpy as np

In [ ]:
# Define the file directory
file_dir = "C:/Users/evanm/Desktop/Classwork/Module 8 - Movies_ETL/Movies_ETL"

In [ ]:
# Save the data as a list of dictionaries
with open(f"{file_dir}/wikipedia.movies.json", mode="r") as file:
    wiki_movies_raw = json.load(file)

In [ ]:
len(wiki_movies_raw)

In [ ]:
# First five records
wiki_movies_raw[:5]

In [ ]:
# Last five records
wiki_movies_raw[-5:]

In [ ]:
# Some records in the middle
wiki_movies_raw[3600:3605]

In [ ]:
# Load the csv files into dataframes
movies_metadata = pd.read_csv(f"{file_dir}/movies_metadata.csv", low_memory=False)
ratings = pd.read_csv(f"{file_dir}/ratings.csv")

In [ ]:
# Load the wiki data into a dataframe
wiki_movies_df = pd.DataFrame(wiki_movies_raw)

In [ ]:
wiki_movies_df.head()

In [ ]:
# Find all the column names
wiki_movies_df.columns.tolist()

In [ ]:
# List comprehension for director and imdb link
wiki_movies = [movie for movie in wiki_movies_raw
              if ("Director" in movie or "Directed by" in movie)
                  and "imdb_link" in movie]
len(wiki_movies)

In [ ]:
# Convert wiki_movies into a dataframe
wiki_movies_new_df = pd.DataFrame(wiki_movies)
wiki_movies_new_df

In [ ]:
# List comprehension for director, imdb link and episodes
wiki_movies = [movie for movie in wiki_movies_raw
              if ("Director" in movie or "Directed by" in movie)
                  and "imdb_link" in movie
                  and "No. of episodes" not in movie]
len(wiki_movies)

In [ ]:
# Convert wiki_movies into a dataframe
wiki_movies_new_df = pd.DataFrame(wiki_movies)
wiki_movies_new_df

In [ ]:
# Create a movie function
def clean_movie(movie):
    
    # Create a non-destructive copy
    movie = dict(movie)
    # Create an empty dictionary to hold alternate titles
    alt_titles = {}
    # Loop through list of alternate title keys
    for key in ["Also know as", "Arabic", "Cantonese", "Chinese", "French",
               "Hangul", "Hebrew", "Hepburn", "Japanese", "Literally",
               "Mandarin", "McCune-Reischauer", "Original title", "Polish",
               "Revised Romanization", "Romanized", "Russian",
               "Simplified", "Traditional", "Yiddish"]:
        # If an alternate title exists, remove it and add to alt_titles
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    # Add the alt_titles dictionary to the movie object
    if len(alt_titles) > 0:
        movie["alt_titles"] = alt_titles
    return movie

In [ ]:
wiki_movies_new_df[wiki_movies_new_df["Arabic"].notnull()]["url"]

In [ ]:
wiki_movies_new_df[wiki_movies_new_df["Hepburn"].notnull()]["Hepburn"]

In [ ]:
# Use list comprehension with clean_movies function
clean_movies = [clean_movie(movie) for movie in wiki_movies]

In [ ]:
# Create a dataframe from clean_movies
wiki_movies_df = pd.DataFrame(clean_movies)
sorted(wiki_movies_df.columns.tolist())